# Redes Neurais

In [ ]:
import pickle

import numpy as np
import pandas as pd

from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict
from sklearn.base import clone, BaseEstimator

from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve, roc_auc_score

import matplotlib as mpl
import matplotlib.pyplot as plt

## MNIST

### Loading Dataset

In [ ]:
with open('../aula01/mnist.pkl', 'rb') as bunch:
    mnist = pickle.load(bunch)

In [ ]:
X, y = mnist['data'].to_numpy(), mnist['target'].to_numpy().astype('uint8')


In [ ]:
X = np.array([x.reshape(28, 28) for x in X])

In [ ]:
X.shape

In [ ]:
split_threshold = 60_000


In [ ]:
X_train, X_test = X[:split_threshold], X[split_threshold:]
y_train, y_test = y[:split_threshold], y[split_threshold:]

## Treinando um classificador binário
Vamos teinar um classificador que verifica apenas se uma imagem
é o número 5 ou não



In [ ]:
y_train_5 = (y_train==5)
y_test_5 = (y_test==5)

In [58]:
y_train_5 = y_train_5.astype(np.float64)
y_test_5 = y_test_5.astype(np.float64)

In [65]:
y_train_5 = np.asarray(y_train_5).astype('float32').reshape((-1,1))
y_test_5 = np.asarray(y_test_5).astype('float32').reshape((-1,1))

In [66]:
y_test_5[22:26]


array([[0.],
       [1.],
       [0.],
       [0.]], dtype=float32)

In [67]:
y_train_5[0]

array([1.], dtype=float32)

In [68]:
import keras

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(2, activation="sigmoid"))

In [69]:
model.compile(loss="binary_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [70]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_7 (Flatten)         (None, 784)               0         
                                                                 
 dense_21 (Dense)            (None, 300)               235500    
                                                                 
 dense_22 (Dense)            (None, 100)               30100     
                                                                 
 dense_23 (Dense)            (None, 2)                 202       
                                                                 
Total params: 265,802
Trainable params: 265,802
Non-trainable params: 0
_________________________________________________________________


In [71]:
history = model.fit(X_train, y_train_5)

ValueError: in user code:

    File "/home/vinicius/aulas/20222/pi/Unip222PI/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/home/vinicius/aulas/20222/pi/Unip222PI/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/vinicius/aulas/20222/pi/Unip222PI/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/home/vinicius/aulas/20222/pi/Unip222PI/venv/lib/python3.8/site-packages/keras/engine/training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/vinicius/aulas/20222/pi/Unip222PI/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "/home/vinicius/a

In [47]:
X_test[0].shape

(28, 28)

In [48]:
model.predict(X_test[:3])

1/1 [==============================] - 0s 47ms/step


array([[0.11462115],
       [0.11462115],
       [0.11462115]], dtype=float32)

In [50]:
y_test_5[:3]

array([0, 0, 0], dtype=int8)

In [51]:
model.predict([X_test[22:25]])

1/1 [==============================] - 0s 43ms/step


array([[0.11462115],
       [0.11462115],
       [0.11462115]], dtype=float32)

In [ ]:
y_test_5[23]

## Cross Validação

In [33]:
# este método nos retorna uma predição de todo o conjunto de treino
# por um classificador que não conhece a parte do cojunto de treino
# que ele está predizendo
y_train_pred = cross_val_predict(model, X_train, y_train_5, cv=2)


TypeError: Cannot clone object '<keras.engine.sequential.Sequential object at 0x7f62e07a6d00>' (type <class 'keras.engine.sequential.Sequential'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.

In [ ]:
confusion_matrix(y_train_5, y_train_pred)

## Precision and Recall and F1

In [ ]:
precision_score(y_train_5, y_train_pred)

In [ ]:
recall_score(y_train_5, y_train_pred)

In [ ]:
f1_score(y_train_5, y_train_pred)